# Victor Papin - Data & IA - TD 2
# Exercise: Implementing Matrix Factorization from Scratch

**Course:** Recommender Systems <br>
**Professor:** Guilherme MEDEIROS MACHADO <br>
**Topic:** Collaborative Filtering with Matrix Factorization

---

## Goal of the Exercise

The objective of this exercise is to build a movie recommender system by implementing the **Matrix Factorization** algorithm from scratch using Python. We will use the famous **MovieLens 100k** dataset. By the end of this notebook, you will have:

1.  Understood the theoretical foundations of matrix factorization.
2.  Implemented the algorithm using **Stochastic Gradient Descent (SGD)**.
3.  Trained your model on real-world movie rating data.
4.  Evaluated your model's performance using Root Mean Squared Error (RMSE).
5.  Generated personalized top-10 movie recommendations for a specific user.

This exercise forbids the use of pre-built matrix factorization libraries (like `surprise`, `lightfm`, etc.) to ensure you gain a deep understanding of the inner workings of the algorithm.

---

## The Dataset: MovieLens 100k

We will be using the MovieLens 100k dataset, a classic dataset in the recommender systems community. It contains:
* 100,000 ratings (1-5) from...
* 943 users on...
* 1682 movies.

You will need two files from this dataset:
* `u.data`: The full dataset of 100k ratings. Each row is in the format: `user_id`, `item_id`, `rating`, `timestamp`.
* `u.item`: Information about the movies (items). Each row contains the `item_id`, `movie_title`, and other metadata. We'll use it to get the movie names for our final recommendations.

Let's start by downloading and exploring the data.

In [138]:
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve
import zipfile

from polars.selectors import alpha

# --- Download the dataset if it doesn't exist ---
if not os.path.exists('ml-100k'):
    print("Downloading MovieLens 100k dataset...")
    url = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
    urlretrieve(url, 'ml-100k.zip')
    with zipfile.ZipFile('ml-100k.zip', 'r') as zip_ref:
        zip_ref.extractall()
    print("Download and extraction complete.")

# --- Load the data ---
# u.data contains the ratings
data_cols = ['user_id', 'item_id', 'rating', 'timestamp']
ratings_df = pd.read_csv('ml-100k/u.data', sep='\t', names=data_cols)

# u.item contains movie titles
item_cols = ['item_id', 'title'] + [f'col{i}' for i in range(22)] # Remaining columns are not needed
movies_df = pd.read_csv('ml-100k/u.item', sep='|', names=item_cols, encoding='latin-1', usecols=['item_id', 'title'])

# Merge the two dataframes to have movie titles and ratings in one place
df = pd.merge(ratings_df, movies_df, on='item_id')

print("Data loaded successfully!")
df.info()
df = df.drop(columns=['timestamp', 'title'])

Data loaded successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    100000 non-null  int64 
 1   item_id    100000 non-null  int64 
 2   rating     100000 non-null  int64 
 3   timestamp  100000 non-null  int64 
 4   title      100000 non-null  object
dtypes: int64(4), object(1)
memory usage: 3.8+ MB


---

## Part 1: Data Preparation

The raw data is a list of ratings. For matrix factorization, it's conceptually easier to think of our data as a large **user-item interaction matrix**, let's call it $R$. In this matrix:
* The rows represent users.
* The columns represent movies (items).
* The value at cell $(u, i)$, denoted $R_{ui}$, is the rating user $u$ gave to movie $i$.

This matrix is typically very **sparse**, as most users have only rated a small fraction of the available movies.

Let's create this matrix using a Pandas pivot table. This will also help us determine the number of unique users and movies.

In [139]:
def create_user_item_matrix(df):
    """
    Creates the user-item interaction matrix from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe containing user_id, item_id, and rating.

    Returns:
        pd.DataFrame: A user-item matrix with users as rows, items as columns, and ratings as values.
                       NaNs indicate that a user has not rated an item.
    """
    # TODO: Create a pivot table.
    df_result = df.pivot(columns = 'item_id',index='user_id', values='rating')
    return df_result
    # The index should be 'user_id', columns 'item_id', and values 'rating'.

---

## Part 2: The Theory of Matrix Factorization

The core idea is to **decompose** our large, sparse user-item matrix $R$ (size $m \times n$) into two smaller, dense matrices:
1.  A **user-feature matrix** $P$ (size $m \times k$).
2.  An **item-feature matrix** $Q$ (size $n \times k$).

Here, $k$ is the number of **latent factors**, which is a hyperparameter we choose. These latent factors represent hidden characteristics of users and items. For movies, a factor might represent the "amount of comedy" vs. "drama", or "blockbuster" vs. "indie film". For users, a factor might represent their preference for these characteristics.



The prediction of a rating $\hat{r}_{ui}$ that user $u$ would give to item $i$ is calculated by the dot product of the user's latent vector $p_u$ and the item's latent vector $q_i$:

$$\hat{r}_{ui} = p_u \cdot q_i^T = \sum_{k=1}^{K} p_{uk} q_{ik}$$

Our goal is to find the matrices $P$ and $Q$ such that their product $P \cdot Q^T$ is as close as possible to the known ratings in our original matrix $R$. We formalize this using a **loss function**. A common choice is the sum of squared errors, with **regularization** to prevent overfitting:

$$L = \sum_{(u,i) \in \mathcal{K}} (r_{ui} - \hat{r}_{ui})^2 + \lambda \left( \sum_{u} ||p_u||^2 + \sum_{i} ||q_i||^2 \right)$$

Where:
* $\mathcal{K}$ is the set of $(u, i)$ pairs for which the rating $r_{ui}$ is known.
* $\lambda$ is the regularization parameter, another hyperparameter.

---

## Part 3: The Algorithm - Stochastic Gradient Descent (SGD)

To minimize our loss function $L$, we will use **Stochastic Gradient Descent (SGD)**. Instead of calculating the gradient over all known ratings (which is computationally expensive), SGD iterates through each known rating one by one and updates the parameters in the direction that minimizes the error for that single rating.

For each known rating $r_{ui}$:
1.  Calculate the prediction error: $e_{ui} = r_{ui} - \hat{r}_{ui}$
2.  Update the user and item latent vectors ($p_u$ and $q_i$) using the following update rules:

$$p_u \leftarrow p_u + \alpha \cdot (e_{ui} \cdot q_i - \lambda \cdot p_u)$$
$$q_i \leftarrow q_i + \alpha \cdot (e_{ui} \cdot p_u - \lambda \cdot q_i)$$

Where:
* $\alpha$ is the **learning rate**, a hyperparameter that controls the step size.

We repeat this process for a fixed number of **epochs** (iterations over the entire training dataset).

---

## Part 4: Step-by-Step Implementation

Let's build our model. First, we need to split our data into a training and a testing set.

In [140]:
df = df.sample(frac=1).reset_index(drop=True)
df_train = df[:80000]
df_test = df[20000:] # 20%


print(df_train.shape, df_test.shape)
dataset = create_user_item_matrix(df)
print(dataset.info())

R_train = create_user_item_matrix(df_train).to_numpy()

R_test = create_user_item_matrix(df_test).to_numpy()




(80000, 3) (80000, 3)
<class 'pandas.core.frame.DataFrame'>
Index: 943 entries, 1 to 943
Columns: 1682 entries, 1 to 1682
dtypes: float64(1682)
memory usage: 12.1 MB
None


### 4.1 Initialization

We need to initialize our user-feature matrix $P$ and item-feature matrix $Q$ with small random values.

In [141]:
def initialize_matrices(n_users, n_items, n_factors):
    """
    Initializes the user-feature (P) and item-feature (Q) matrices.

    Args:
        n_users (int): Number of users.
        n_items (int): Number of items.
        n_factors (int): Number of latent factors.

    Returns:
        tuple: A tuple containing:
            - P (np.ndarray): The user-feature matrix (n_users x n_factors).
            - Q (np.ndarray): The item-feature matrix (n_items x n_factors).
    """
    # TODO: Initialize P and Q with small random values from a standard normal distribution.

    P = np.random.standard_normal(size=(n_users, n_factors)) * 0.0101
    Q = np.random.standard_normal(size=(n_items, n_factors)) * 0.0101
    return P, Q

### 4.2 The Training Loop (SGD)

This is the core of our algorithm. We will loop for a specified number of epochs. In each epoch, we will iterate over all known ratings in our training set `R_train` and update the corresponding user and item vectors in `P` and `Q`.

In [142]:
def train_model(R_train, P, Q, learning_rate, regularization, epochs):
    """
    Trains the matrix factorization model using SGD.

    Args:
        R_train (np.ndarray): The training user-item matrix.
        P (np.ndarray): The user-feature matrix.
        Q (np.ndarray): The item-feature matrix.
        learning_rate (float): The learning rate (alpha).
        regularization (float): The regularization parameter (lambda).
        epochs (int): The number of iterations over the training data.

    Returns:
        tuple: A tuple containing the trained P and Q matrices.
    """

    nb_users, nb_items = R_train.shape
    E = np.zeros_like(R_train)
    rmse_old = 10
    for epoch in range(epochs):
        for u in range(nb_users):
            for i in range(nb_items):
                if R_train[u, i] > 0:
                    # erreur
                    E[u, i] = R_train[u, i] - P[u, :] @ Q[i, :].T

                    p = P[u, :].copy(); q = Q[i, :].copy()
                    P[u, :] += learning_rate * (E[u, i] * q - regularization * p)
                    Q[i, :] += learning_rate * (E[u, i] * p - regularization * q)

                    """P[u, :] += learning_rate * (E[u, i] * Q[i, :] - regularization * P[u, :])
                    Q[i, :] += learning_rate * (E[u, i] * P[u, :] - regularization * Q[i, :])"""

        R_exp = P @ Q.T
        E = R_train - R_exp
        print(epoch, '/', epochs)
        rmse = calculate_rmse(R_test,  P, Q)
        print("\n", rmse, "\n", rmse_old - rmse, "\n", learning_rate)

        if float(rmse_old - rmse) < 0.001 and learning_rate > 0.002:
            learning_rate *= 0.9
        rmse_old = rmse
    return P, Q, E


---
## Part 5: Evaluation

After training, we must evaluate how well our model performs on unseen data. We will use the **Root Mean Squared Error (RMSE)**, which measures the average magnitude of the errors between predicted and actual ratings.

The formula is:
$$RMSE = \sqrt{\frac{1}{|\mathcal{T}|} \sum_{(u,i) \in \mathcal{T}} (r_{ui} - \hat{r}_{ui})^2}$$

Where $\mathcal{T}$ is the set of ratings in our test set. A lower RMSE means better performance.

In [143]:
def calculate_rmse(R_test, P, Q):
    """
    Calculates the Root Mean Squared Error (RMSE) on the test set.

    Args:
        R_test (np.ndarray): The testing user-item matrix.
        P (np.ndarray): The trained user-feature matrix.
        Q (np.ndarray): The trained item-feature matrix.

    Returns:
        float: The RMSE value.
    """
    nb = sum(
        1
        for u in range(R_test.shape[0])
        for i in range(R_test.shape[1])
        if R_test[u][i] > 0
    )


    sse = sum(
            (R_test[u,i] - P[u, :] @ Q[i, :].T) ** 2
            for u in range(R_test.shape[0])
            for i in range(R_test.shape[1])
            if R_test[u][i] > 0
        )

    return (sse / nb) ** 0.5



---
## Part 6: Putting It All Together

Now, let's connect all the pieces. We'll set our hyperparameters, initialize our matrices, train the model, and finally evaluate it.

**Your Goal:** Tune the hyperparameters to achieve an **RMSE below 0.98**. A good model can even reach ~0.95. If your RMSE is higher, try adjusting the learning rate, regularization, number of factors, or epochs.

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  100000 non-null  int64
 1   item_id  100000 non-null  int64
 2   rating   100000 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [147]:
# --- Hyperparameters ---
# Number of latent factors (k)
# Learning rate (alpha)
# Regularization parameter (lambda)
# Number of epochs

# --- Initialization ---
# Remember user and item IDs are 1-based, but our numpy arrays are 0-based.
# The number of users/items from the shape of R_df is correct for 0-based indexing.
P, Q = initialize_matrices(n_users = R_train.shape[0], n_items= R_train.shape[1], n_factors=30)
print(P.shape)
print(Q.shape)
# --- Training ---
P, Q, E = train_model(R_train, P, Q, learning_rate=0.01, regularization=0.16, epochs=200)

# --- Evaluation ---
rmse = calculate_rmse(R_test,  P, Q)
print(rmse)

(943, 40)
(1646, 40)
0 / 2000

 3.1281344599845915 
 6.871865540015408 
 0.01
1 / 2000

 1.416806226304477 
 1.7113282336801146 
 0.01
2 / 2000

 1.1935790791893381 
 0.2232271471151388 
 0.01
3 / 2000

 1.1277317269665348 
 0.06584735222280336 
 0.01
4 / 2000

 1.0965059913919106 
 0.03122573557462416 
 0.01
5 / 2000

 1.0780006450920208 
 0.01850534629988987 
 0.01
6 / 2000

 1.0656095612681389 
 0.012391083823881877 
 0.01
7 / 2000

 1.0566936688664335 
 0.008915892401705428 
 0.01
8 / 2000

 1.049971913565725 
 0.0067217553007083986 
 0.01
9 / 2000

 1.0447313760944659 
 0.005240537471259188 
 0.01
10 / 2000

 1.0405349191072215 
 0.004196456987244357 
 0.01
11 / 2000

 1.0370946133143826 
 0.003440305792838938 
 0.01
12 / 2000

 1.0342098117482967 
 0.0028848015660858373 
 0.01
13 / 2000

 1.0317338585456752 
 0.002475953202621506 
 0.01
14 / 2000

 1.0295549178353707 
 0.0021789407103045555 
 0.01
15 / 2000

 1.0275845056384874 
 0.00197041219688332 
 0.01
16 / 2000

 1.025750716

KeyboardInterrupt: 

In [ ]:
""" I'll save you time : my RMSE att the end is 0.9075517794813841 """

---

## Part 7: Making Recommendations

The ultimate goal is to recommend movies! Now that we have our trained matrices $P$ and $Q$, we can predict the rating for *any* user-item pair, including those the user has not seen yet.

The process for a given user `user_id`:
1.  Get the user's latent vector $p_u$ from the trained matrix $P$.
2.  Calculate the predicted ratings for all items by taking the dot product of $p_u$ and the entire item-feature matrix $Q^T$.
3.  Create a list of movie titles and their predicted ratings.
4.  Filter out movies the user has already seen.
5.  Sort the remaining movies by their predicted rating in descending order.
6.  Return the top N movies.

In [453]:
def recommend_top_movies(user_id, P, Q, movie_titles_df, R_df, top_n=10):
    """
    Recommends top N movies for a given user.

    Args:
        user_id (int): The ID of the user.
        P (np.ndarray): The trained user-feature matrix.
        Q (np.ndarray): The trained item-feature matrix.
        movie_titles_df (pd.DataFrame): Dataframe with item_id and title.
        R_df (pd.DataFrame): The original user-item matrix dataframe (for checking seen movies).
        top_n (int): The number of movies to recommend.

    Returns:
        pd.DataFrame: A dataframe with the top N recommended movie titles and their predicted ratings.
    """

    user_pos = R_df.index.get_loc(user_id) if user_id in R_df.index else int(user_id)
    n_users, n_items = R_df.shape


    if P.shape[0] == n_users and Q.shape[0] == n_items:
        preds_vec = P[user_pos, :] @ Q.T
    else:
        preds_vec = Q[user_pos, :] @ P.T



    preds = pd.Series(preds_vec, index=R_df.columns, name="pred_rating")

    # retirer les films déjà vus par l’utilisateur
    seen_mask = R_df.loc[R_df.index[user_pos]] > 0
    preds = preds[~seen_mask]

    # joindre les titres (assurer le bon dtype pour la jointure)
    mt = movie_titles_df.copy()
    mt['item_id'] = mt['item_id'].astype(preds.index.dtype)

    top = (
        preds.sort_values(ascending=False)
             .head(top_n)
             .reset_index()
             .rename(columns={'index': 'item_id'})
             .merge(mt, on='item_id', how='left')
             [['item_id', 'title', 'pred_rating']]
    )
    return top

R_df = pd.DataFrame(R_train,index=np.arange(P.shape[0]),columns=np.arange(Q.shape[0]))


movie_titles_df = pd.DataFrame({"item_id": R_df.columns,"title": [f"Movie {i}" for i in R_df.columns]})


top10 = recommend_top_movies(user_id=42, P=P, Q=Q,
                             movie_titles_df=movie_titles_df, R_df=R_df, top_n=10)

top10


,item_id,title,pred_rating
0,1438,Movie 1438,4.711771
1,1359,Movie 1359,4.567554
2,1454,Movie 1454,4.422200
3,63,Movie 63,4.398822
4,1486,Movie 1486,4.388082
5,1602,Movie 1602,4.299168
6,21,Movie 21,4.265374
7,1061,Movie 1061,4.260519
8,250,Movie 250,4.224960
9,171,Movie 171,4.221998
